In [1]:
%load_ext lab_black
import subprocess, os
import pandas as pd

# Input and Parameters

In [2]:
# MODIFY THIS CELL

# system control
env_bin = "/home/coco/miniconda3/envs/cut_run/bin/"
num_cpus = 30

# sample name: give more detailed, cell type, experiment, targets etc...
out_prefix = "E14_Brg1FV_ATAC_0hr_Rep2"

# FASTQ input paths
R1 = "HL31_0404_S62_L004_R1_001.fastq.gz"
R2 = "HL31_0404_S62_L004_R2_001.fastq.gz"

# adaptor sequence for Trimmomatic
adaptor_seq_fa = "/Extension_HDD2/Hanbin/ES_Cell/E14/ATAC/NexteraPE-PE.fa"

# bowtie2 mapping setup
genome_index = "/home/software/bowtie2-2.2.9/genome/mm10/mm10"

# fragment size
low, high = 0, 200

# server for bigwig
url = "http://unzip.4d-genome.com:8080/E14_Brg1FV_ATAC/"
webdir = "/usr/local/apache2/htdocs/E14_Brg1FV_ATAC/"

# MACS2 call peak parameters
# keep_dup = 3
macs2_genome = "mm"

# Trim Adaptor PE Mode

In [3]:
R1_paired_trimmed = f"{out_prefix}_1.pair.fastq"
R2_paired_trimmed = f"{out_prefix}_2.pair.fastq"
R1_unpaired_trimmed = f"{out_prefix}_1.unpair.fastq"
R2_unpaired_trimmed = f"{out_prefix}_2.unpair.fastq"

if R1.endswith("gz"):
    R1_paired_trimmed += ".gz"
    R2_paired_trimmed += ".gz"
    R1_unpaired_trimmed += ".gz"
    R2_unpaired_trimmed += ".gz"

print(f"Trimming {R1} and {R2}")
trim_adaptor = subprocess.run(
    [
        env_bin + "trimmomatic",
        "PE",
        "-threads",
        str(num_cpus),
        "-phred33",
        R1,
        R2,
        R1_paired_trimmed,
        R1_unpaired_trimmed,
        R2_paired_trimmed,
        R2_unpaired_trimmed,
        f"ILLUMINACLIP:{adaptor_seq_fa}:2:15:4:4:true",
        "LEADING:20",
        "TRAILING:20",
        "SLIDINGWINDOW:4:15",
        "MINLEN:25",
    ],
)
if trim_adaptor.returncode != 0:
    print(trim_adaptor.stderr.decode())

Trimming HL31_0404_S62_L004_R1_001.fastq.gz and HL31_0404_S62_L004_R2_001.fastq.gz


Picked up _JAVA_OPTIONS: -Xmx110G
TrimmomaticPE: Started with arguments:
 -threads 30 -phred33 HL31_0404_S62_L004_R1_001.fastq.gz HL31_0404_S62_L004_R2_001.fastq.gz E14_Brg1FV_ATAC_0hr_Rep2_1.pair.fastq.gz E14_Brg1FV_ATAC_0hr_Rep2_1.unpair.fastq.gz E14_Brg1FV_ATAC_0hr_Rep2_2.pair.fastq.gz E14_Brg1FV_ATAC_0hr_Rep2_2.unpair.fastq.gz ILLUMINACLIP:/Extension_HDD2/Hanbin/ES_Cell/E14/ATAC/NexteraPE-PE.fa:2:15:4:4:true LEADING:20 TRAILING:20 SLIDINGWINDOW:4:15 MINLEN:25
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Input Read Pairs: 35747914 Both Surviving: 34947569 (97.76%) Forward Only Surviving:

# Bowtie2 Mappping

In [4]:
sam_all_out = f"{out_prefix}.sam"
mapping_stats = ""
with open(sam_all_out, "w") as o:
    bwt2_mapping = subprocess.Popen(
        [
            env_bin + "bowtie2",
            "--very-sensitive",
            "-x",
            genome_index,
            "-p",
            str(num_cpus),
            "-I",
            "10",
            "-X",
            "700",
            "--dovetail",
            "--phred33",
            "-1",
            R1_paired_trimmed,
            "-2",
            R2_paired_trimmed,
        ],
        stderr=subprocess.PIPE,
        stdout=o,
    )

    # capture mapping stats
    for line in iter(bwt2_mapping.stderr.readline, b""):
        if line.startswith(b"WARN"):
            continue
        else:
            mapping_stats += line.decode()

    bwt2_mapping.wait()

In [5]:
print(f"Mapped {sam_all_out}")
print(mapping_stats)

Mapped E14_Brg1FV_ATAC_0hr_Rep2.sam
34947569 reads; of these:
  34947569 (100.00%) were paired; of these:
    986084 (2.82%) aligned concordantly 0 times
    22746202 (65.09%) aligned concordantly exactly 1 time
    11215283 (32.09%) aligned concordantly >1 times
    ----
    986084 pairs aligned concordantly 0 times; of these:
      204472 (20.74%) aligned discordantly 1 time
    ----
    781612 pairs aligned 0 times concordantly or discordantly; of these:
      1563224 mates make up the pairs; of these:
        964131 (61.68%) aligned 0 times
        324314 (20.75%) aligned exactly 1 time
        274779 (17.58%) aligned >1 times
98.62% overall alignment rate



# Fragment Size Distribution

In [6]:
bam_all_out = f"{out_prefix}.sorted.bam"
subprocess.run(
    f"{env_bin}samtools sort {sam_all_out} -O bam -o {bam_all_out} -@ {num_cpus}",
    shell=True,
)
subprocess.run(f"{env_bin}samtools index {bam_all_out} -@ {num_cpus}", shell=True)

[bam_sort_core] merging from 0 files and 30 in-memory blocks...


CompletedProcess(args='/home/coco/miniconda3/envs/cut_run/bin/samtools index E14_Brg1FV_ATAC_0hr_Rep2.sorted.bam -@ 30', returncode=0)

In [7]:
!bamPEFragmentSize -hist fragmentSize.png -T "Fragment size of PE ATAC data" --maxFragmentLength 1000 -b $bam_all_out --samplesLabel $out_prefix



Sample label: E14_Brg1FV_ATAC_0hr_Rep2
Sample size: 34098

Fragment lengths:
Min.: 0.0
1st Qu.: 78.0
Mean: 191.4351868144759
Median: 132.0
3rd Qu.: 257.0
Max.: 700.0
Std: 151.8264211652476
MAD: 68.0
Len. 10%: 55.0
Len. 20%: 70.0
Len. 30%: 86.0
Len. 40%: 107.0
Len. 60%: 174.0
Len. 70%: 225.0
Len. 80%: 302.0
Len. 90%: 430.0
Len. 99%: 653.0


Read lengths:
Sample size: 34098

Min.: 25.0
1st Qu.: 75.0
Mean: 87.30696815062467
Median: 101.0
3rd Qu.: 101.0
Max.: 101.0
Std: 20.185209045584696
MAD: 0.0
Len. 10%: 53.0
Len. 20%: 67.0
Len. 30%: 82.0
Len. 40%: 100.0
Len. 60%: 101.0
Len. 70%: 101.0
Len. 80%: 101.0
Len. 90%: 101.0
Len. 99%: 101.0



# Deeptools Create BigWig

In [8]:
bw = f"{out_prefix}.{low}_{high}.bigWig"
print(f"Generating {bw}")

# make bigWig
subprocess.run(
    f"bamCoverage -b {bam_all_out} -o {bw} -p {num_cpus} -bs 10 --normalizeUsing CPM  --minMappingQuality 30 --minFragmentLength {low} --maxFragmentLength {high}",
    shell=True,
)

# setup the link for bigWig
import random


def create_bw_track_controller(bw_path, webdir, host, track_name=None, color=None):
    if track_name is None:
        track_name = bw_path.split("/")[-1]

    if color is None:
        color = f"{random.randrange(0, 256)},{random.randrange(0, 256)},{random.randrange(0, 256)}"

    # host url for the webdir
    subprocess.run(["ln", "-s", os.path.abspath(bw_path), os.path.abspath(webdir)])
    track_ctl = (
        "track type=bigWig "
        + f"name={track_name} "
        + f"bigDataUrl={host}/{track_name} "
        + f"color={color} "
        + 'visibility=full yLineOnOff=on autoScale=on yLineMark="0.0" alwaysZero=on graphType=bar maxHeightPixels=128:75:11 windowingFunction=maximum smoothingWindow=off'
    )

    with open(f"{bw_path}.track_control.txt", "w") as o:
        o.write(track_ctl)

    bw = bw_path.split("/")[-1]
    with open(f"{webdir}/{bw}.track_control.txt", "w") as o:
        o.write(
            "#"
            + " ".join(["ln", "-s", os.path.abspath(bw_path), os.path.abspath(webdir)])
            + "\n"
        )
        o.write(track_ctl)


create_bw_track_controller(bw, webdir, url)

Generating E14_Brg1FV_ATAC_0hr_Rep2.0_200.bigWig
Due to filtering, 40.771523126824064% of the aforementioned alignments will be used 28104221.46055771


normalization: CPM
bamFilesList: ['E14_Brg1FV_ATAC_0hr_Rep2.sorted.bam']
binLength: 10
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 30
verbose: False
region: None
bedFile: None
minMappingQuality: 30
ignoreDuplicates: False
chrsToSkip: []
stepSize: 10
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 200
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 200


# Keep Fragment by Size and Filter MAPQ

In [9]:
filtered_frag_bed = f"{out_prefix}.mapq30_{low}_{high}_fragment.bed"
sort_by_name = subprocess.Popen(
    f"{env_bin}samtools sort {sam_all_out} -O bam -@ {num_cpus} -n",
    shell=True,
    stdout=subprocess.PIPE,
)

bedtools_to_bedpe = subprocess.Popen(
    f"bedtools bamtobed -bedpe -i stdin",
    shell=True,
    stdin=sort_by_name.stdout,
    stdout=subprocess.PIPE,
)

with open(filtered_frag_bed, "w") as w:
    subprocess.run(
        'awk \'($1 == $4) && ($8 >= 30) && ($6 - $2 >=0) && ($6 - $2 <= 200) {print $1"\\t"$2"\\t"$6}\'',
        shell=True,
        stdin=bedtools_to_bedpe.stdout,
        stdout=w,
    )

[bam_sort_core] merging from 0 files and 30 in-memory blocks...


In [10]:
subprocess.run(["wc", "-l", filtered_frag_bed])

16109757 E14_Brg1FV_ATAC_0hr_Rep2.mapq30_0_200_fragment.bed


CompletedProcess(args=['wc', '-l', 'E14_Brg1FV_ATAC_0hr_Rep2.mapq30_0_200_fragment.bed'], returncode=0)

# MACS2 Call Peaks

In [11]:
macs2_out_prefix = f"{out_prefix}.mapq30_{low}_{high}.macs2"

with open(f"{macs2_out_prefix}.log", "w") as log:
    macs2_cmd = [
        f"{env_bin}macs2",
        "callpeak",
        "-t",
        filtered_frag_bed,
        "-g",
        macs2_genome,
        "-f",
        "BEDPE",
        "-n",
        macs2_out_prefix,
        "-q",
        str(0.01),
        "-B",
        "--SPMR",
        "--keep-dup",
        "auto",
    ]

    macs2_callpeaks = subprocess.Popen(
        macs2_cmd,
        stderr=log,
    )
    macs2_callpeaks.wait()

# peak number
subprocess.run(["wc", "-l", f"{macs2_out_prefix}_peaks.narrowPeak"])

40514 E14_Brg1FV_ATAC_0hr_Rep2.mapq30_0_200.macs2_peaks.narrowPeak


CompletedProcess(args=['wc', '-l', 'E14_Brg1FV_ATAC_0hr_Rep2.mapq30_0_200.macs2_peaks.narrowPeak'], returncode=0)